In [1]:
import numpy as np
import tensorflow as tf 

In [2]:
class HMM(object):
    def __init__(self, initial_prob, trans_prob, obs_prob):
        self.N = np.size(initial_prob)
        self.initial_prob = initial_prob
        self.trans_prob = trans_prob
        self.emission = obs_prob
        assert self.initial_prob.shape == (self.N, 1)
        assert self.trans_prob.shape == (self.N, self.N)
        assert obs_prob.shape[0] == self.N
        self.obs_idx = 0
        self.fwd = 0.
        
    def get_emission(self, obs_idx):
        slice_location = [0, obs_idx]
        num_rows = tf.shape(self.emission)[0]
        slice_shape = [num_rows, 1]
        return tf.slice(self.emission, slice_location, slice_shape)
    
    def forward_init_op(self, obs_idx):
        self.obs_idx = obs_idx
        obs_prob = self.get_emission(self.obs_idx)
        fwd = tf.multiply(self.initial_prob, obs_prob)
        return fwd
    
    def forward_op(self, obs_idx, fwd):
        self.obs_idx = obs_idx
        self.fwd = fwd
        transitions = tf.matmul(self.fwd,
        tf.transpose(self.get_emission(self.obs_idx)))
        weighted_transitions = transitions * self.trans_prob
        fwd = tf.reduce_sum(weighted_transitions, 0)
        return tf.reshape(fwd, tf.shape(self.fwd))

In [3]:
def forward_algorithm(hmm, observations):
    fwd = hmm.forward_init_op(observations[0])
    for t in range(1, len(observations)):
        fwd = hmm.forward_op(observations[t],fwd)
    prob = tf.reduce_sum(fwd)
    return prob

In [4]:
initial_prob = np.array([[0.6],[0.4]])
trans_prob = np.array([[0.7, 0.3],
                       [0.4, 0.6]])
obs_prob = np.array([[0.1, 0.4, 0.5],
                     [0.6, 0.3, 0.1]])
hmm = HMM(initial_prob=initial_prob, trans_prob=trans_prob, obs_prob=obs_prob)
observations = [0, 1, 1, 2, 1]

prob = forward_algorithm(hmm, observations)
print('Probability of observing {} is {}'.format(observations, prob))

Probability of observing [0, 1, 1, 2, 1] is 0.004540300799999999


In [5]:
print('Emission Shape', np.shape(obs_prob))

Emission Shape (2, 3)


In [8]:
# this walks through the loop unfolding to learn more about how the HMM is actually doing the comuptation in this 
# simple weather example

# basically in the first iteration to initialize, the HMM takes the first observation index, 0, aka 'walk'
# and multiplies the probabilities of walking when it is raining / sunny times by the initial probabilities of it being rainy
# or sunny to get a composite probability for it being rainy or sunny and having walked

# then the next step steps through the rest of the observations, 'shopping', 'shopping', 'cleaning', 'shopping', and does
# the following

# for 'shopping', it takes the emission probability of seeing shopping in each state at that point, and multiplies it by the existing
# forward model, then applies the weighted transition probabilities in each actual state (Rainy or Sunny), 
# through multiplication, and then accumulates them (through reduce_sum) to get the cumulitative probability of being 
# in either state given the observations thus far ('walk', 'shopping')

# it does the same thing for the next observations, 'shopping', 'cleaning' and 'shopping' to get the cumultative probability
# of being in either state

# the final answer is the cumultation of either probability, so the likelihood of that sequence of events happening, 
# but not necessarily the highest probability state you are in

slice_location = [0, 0]
num_rows = tf.shape(obs_prob)[0]
num_rows_eval = num_rows.numpy()
slice_shape = [num_rows, 1]
fwd0 = tf.slice(obs_prob, slice_location, slice_shape)
fwd0_eval = fwd0.numpy()
fwd0 = tf.multiply(initial_prob, fwd0)
fwd0_eval = fwd0.numpy()
transitions1 = tf.matmul(fwd0, tf.transpose(np.array([[0.4], [0.3]])))
transitions1_e = transitions1.numpy()
weighted_transitions1 = transitions1 * trans_prob
fwd1 = tf.reduce_sum(weighted_transitions1, 0)
fwd1_prs_e = fwd1.numpy()
fwd1 = tf.reshape(fwd1, tf.shape(fwd0))
fwd1_eval = fwd1.numpy()

transitions2 = tf.matmul(fwd1, tf.transpose(np.array([[0.4], [0.3]])))
weighted_transitions2 = transitions2 * trans_prob
fwd2 = tf.reduce_sum(weighted_transitions2, 0)
fwd2_prs_e = fwd2.numpy()
fwd2 = tf.reshape(fwd2, tf.shape(fwd1))
fwd2_eval = fwd2.numpy()


transitions3 = tf.matmul(fwd2, tf.transpose(np.array([[0.5], [0.1]])))
weighted_transitions3 = transitions3 * trans_prob
fwd3 = tf.reduce_sum(weighted_transitions3, 0)
fwd3_prs_e = fwd3.numpy()
fwd3 = tf.reshape(fwd3, tf.shape(fwd2))
fwd3_eval = fwd3.numpy()

transitions4 = tf.matmul(fwd3, tf.transpose(np.array([[0.4], [0.3]])))
weighted_transitions4 = transitions4 * trans_prob
fwd4 = tf.reduce_sum(weighted_transitions4, 0)
fwd4_prs_e = fwd4.numpy()
fwd4 = tf.reshape(fwd4, tf.shape(fwd3))
fwd4_eval = fwd4.numpy()

final_prob = tf.reduce_sum(fwd4)
final_prob_e = final_prob.numpy()

In [9]:
print('Num Rows', num_rows_eval)

Num Rows 2


In [10]:
print(fwd0_eval)
#print(transitions1_e)
#print(trans_prob)
#print(transitions1_e*trans_prob)
print(fwd1_prs_e)
print(fwd1_eval)
print(fwd2_prs_e)
print(fwd2_eval)
print(fwd3_prs_e)
print(fwd3_eval)
print(fwd4_prs_e)
print(fwd4_eval)
print(final_prob_e)

[[0.06]
 [0.24]]
[0.0552 0.0486]
[[0.0552]
 [0.0486]]
[0.023232 0.013716]
[[0.023232]
 [0.013716]]
[0.0108744  0.00151992]
[[0.0108744 ]
 [0.00151992]]
[0.00328802 0.00125228]
[[0.00328802]
 [0.00125228]]
0.004540300799999999
